In [1]:
sc

In [2]:
sc.applicationId

u'application_1522648856070_0850'

## Required Libraries

In [3]:
import sys
sys.path.append("/usr/lib/python2.7/site-packages")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from pyspark.sql.functions import year, month, dayofmonth, hour
from pyspark.sql.functions import col, asc, desc,log
import pyspark.sql.functions as F

from pyspark.sql.functions import broadcast

In [4]:
# For displaying multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load bus tracking data for a month

In [5]:
sqlContext.sql("use bmtcvts")
sqlContext.sql("show tables").show()

DataFrame[]

+--------------------+-----------+
|           tableName|isTemporary|
+--------------------+-----------+
|parquet_route_bus...|      false|
|parquet_route_bus...|      false|
|parquet_vts_parse...|      false|
|parquet_vts_parse...|      false|
|parquet_vts_parse...|      false|
|   vts_apr17_parquet|      false|
|   vts_aug16_parquet|      false|
|   vts_aug17_parquet|      false|
|   vts_dec16_parquet|      false|
|   vts_dec17_parquet|      false|
|   vts_feb17_parquet|      false|
|   vts_jan17_parquet|      false|
|   vts_jul16_parquet|      false|
|   vts_jul17_parquet|      false|
|   vts_jun17_parquet|      false|
|   vts_mar17_parquet|      false|
|   vts_may17_parquet|      false|
|   vts_nov16_parquet|      false|
|   vts_nov17_parquet|      false|
|   vts_oct16_parquet|      false|
+--------------------+-----------+
only showing top 20 rows



In [6]:
# Laod the VTS data for the duration of jul16 - jun 17
# We shall begin with data for only October 2017
months = ["jan17", "feb17", "mar17", "apr17", "may17", "jun17",
         "jul17", "aug17", "sep17", "oct17", "nov17", "dec17"]

all_months = ["vts_" + mth + "_parquet" for mth in months]
all_months

vts_df = []
for index in range(len(all_months)):
    sql_query = "select * from " + all_months[index]
    vts_df.append(sqlContext.sql(sql_query))

['vts_jan17_parquet',
 'vts_feb17_parquet',
 'vts_mar17_parquet',
 'vts_apr17_parquet',
 'vts_may17_parquet',
 'vts_jun17_parquet',
 'vts_jul17_parquet',
 'vts_aug17_parquet',
 'vts_sep17_parquet',
 'vts_oct17_parquet',
 'vts_nov17_parquet',
 'vts_dec17_parquet']

In [7]:
len(vts_df)
type(vts_df)
type(vts_df[0])

12

list

pyspark.sql.dataframe.DataFrame

In [8]:
# Combine the data from all months into a single dataframe
# Declare a counter to store the number of VTS rows
# in each month
counter = []

# Append all of the data frames into a single DF, via union
vts_main_df = vts_df[0]
counter.append(vts_df[0].count())
for index in range(len(vts_df)-1):
    counter.append(vts_df[index+1].count())
    vts_main_df = vts_main_df.union(vts_df[index+1])

In [9]:
counter

[1067855533,
 976218546,
 1080027417,
 999717643,
 1005573387,
 984483441,
 1012715056,
 1034188924,
 1025564723,
 1016240615,
 1023529365,
 666380862]

In [10]:
vts_main_filtered_df = vts_main_df.select("id", col("device_id").alias("VTS_device_id"), \
                                          "ign_status", "acc_distance", "ist_date", "lat", \
                                          "longitude", "vehicle_direction")

In [11]:
vts_main_filtered_df.show(2)

+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
|         id|VTS_device_id|ign_status|acc_distance|            ist_date|      lat|longitude|vehicle_direction|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
|12132590050|    150814189|         1|    48644366|2017-01-01 02:09:...|13.100615|77.594879|            273.0|
|12132590052|    150220564|         1|    48490776|2017-01-01 02:06:...|13.137562|77.479797|             84.0|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
only showing top 2 rows



In [12]:
vts_main_nonzero_df = vts_main_filtered_df.filter((col("lat") != 0.0) | (col("longitude") != 0.0))

In [13]:
vts_main_nonzero_count = vts_main_nonzero_df.count()
vts_main_nonzero_df.show(2)
vts_main_nonzero_count

+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
|         id|VTS_device_id|ign_status|acc_distance|            ist_date|      lat|longitude|vehicle_direction|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
|12132590050|    150814189|         1|    48644366|2017-01-01 02:09:...|13.100615|77.594879|            273.0|
|12132590052|    150220564|         1|    48490776|2017-01-01 02:06:...|13.137562|77.479797|             84.0|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
only showing top 2 rows



11375513408

## Load static data

In [14]:
sqlContext.sql("use bmtc")

# Get the route map
route_map_df = sqlContext.sql("select route_id,start_bus_stop_id,end_bus_stop_id,\
                                      distance,time_to_travel,bus_stop_order,status \
                               from route_map")

# Get the route_point
route_point_df = sqlContext.sql("select route_id, route_order, bus_stop_id from route_point")

bus_stop_df = sqlContext.sql("select bus_stop_id,bus_stop_name,latitude_current,longitude_current from bus_stop")

# Drop corrupted locations
bus_stop_df = bus_stop_df.na.drop(subset=["latitude_current"])
bus_stop_df = bus_stop_df.na.drop(subset=["longitude_current"])

# Join the bus stop ID with lat,long
route_point_joined_df = route_point_df.join(bus_stop_df,\
                                            ["bus_stop_id"],\
                                            "left_outer")

form_four_df = sqlContext.sql("select form_four_id,form_four_name,schedule_number_id,\
                                      schedule_number_name,no_of_trips,start_time,\
                                      route_id,route_number,toll_zone,\
                                      area_limit,total_km,total_dead_km,\
                                      actual_km,total_running_time,total_break_time,\
                                      total_steering_time,spread_over_hours,ot_hours \
                               from form_four")

schedule_df = sqlContext.sql('select * from schedule')

schedule_df = schedule_df.select("schedule_id","schedule_number","depot_id","route_id","schedule_type")

schedule_details_df = sqlContext.sql('select * from schedule_details')

schedule_details_df = schedule_details_df.select("schedule_details_id","form_four_id","schedule_number","number_of_trips",\
                           "trip_number","trip_type","start_point","end_point","route_number_id",\
                           "route_number","route_direction","distance","start_time","end_time",\
                           "running_time","break_type_id","shift_type_id","is_dread_trip")

DataFrame[]

In [15]:
sqlContext.sql("use bmtcwaybill")

# Get the waybill details, and clean it
waybill_trip_details_df = sqlContext.sql("select * from waybill_trip_details")

waybill_trip_details_df = waybill_trip_details_df.select("id","waybill_id","duty_dt",
                                                         col("device_id").alias("WB_device_id"),
                                          "schedule_type_id","schedule_no","schedule_name","service_type",
                                          "service_name","trip_number","start_point","start_bus_stop_name",
                                          "end_point","end_bus_stop_name","route_id","route_no","distance",
                                          "start_time","act_start_time","etm_start_time","end_time","act_end_time",
                                          "etm_end_time","running_time","is_dread_trip")

DataFrame[]

In [16]:
# Filter waybill for the the Year 2017
waybill_trip_details_filtered_df = waybill_trip_details_df.filter(year(waybill_trip_details_df.duty_dt) == 2017)

## Filter rows for Volvo services only

In [17]:
volvo_waybill_df = waybill_trip_details_filtered_df.filter(col("schedule_name").like("V%"))
volvo_waybill_count = volvo_waybill_df.count()
volvo_waybill_count

2708934

In [18]:
volvo_device_id_list = volvo_waybill_df.select("WB_device_id").rdd.map(lambda x: x[0]).distinct().collect()
volvo_device_id_count = len(volvo_device_id_list)
volvo_device_id_count

1269

## Filter VTS data for Volvo services

In [19]:
vts_volvo_df = vts_main_nonzero_df.where(col("VTS_device_id").isin(volvo_device_id_list))
vts_volvo_df.show(3)
vts_volvo_count = vts_volvo_df.count()
vts_volvo_count

+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
|         id|VTS_device_id|ign_status|acc_distance|            ist_date|      lat|longitude|vehicle_direction|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
|12132590056|    150220781|         0|    39713995|2017-01-01 02:09:...|12.999368|77.675102|            255.0|
|12132590069|    150221435|         0|    44149312|2017-01-01 02:09:...|13.004257|77.692299|            188.0|
|12132590072|    150221408|         0|    47214336|2017-01-01 02:09:...|12.977072| 77.72702|            118.0|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+
only showing top 3 rows



1576992211

## Implementing grid approach for Volvo services only

In [20]:
route_point_joined_df.show(2)
bus_stop_df.show(2)

+-----------+--------+-----------+--------------------+----------------+-----------------+
|bus_stop_id|route_id|route_order|       bus_stop_name|latitude_current|longitude_current|
+-----------+--------+-----------+--------------------+----------------+-----------------+
|        160|     809|          1|Kempegowda Bus St...|     12.97751447|      77.57178022|
|       5841|     809|          2|       KBS Connector|     12.97752049|      77.57099145|
+-----------+--------+-----------+--------------------+----------------+-----------------+
only showing top 2 rows

+-----------+-----------------+----------------+-----------------+
|bus_stop_id|    bus_stop_name|latitude_current|longitude_current|
+-----------+-----------------+----------------+-----------------+
|          3|        Kodihalli|     12.96004656|      77.64719738|
|          4|Vijayanagara TTMC|     12.96572391|      77.53510224|
+-----------+-----------------+----------------+-----------------+
only showing top 2 rows



In [21]:
bus_stop_count = bus_stop_df.count()
bus_stop_count

11818

In [22]:
bus_stop_distinct_df = bus_stop_df.select("latitude_current","longitude_current").distinct()
bus_stop_distinct_df.show(2)
bus_stop_distinct_count = bus_stop_distinct_df.count()
bus_stop_distinct_count

+----------------+-----------------+
|latitude_current|longitude_current|
+----------------+-----------------+
|     12.97582743|      77.60564176|
|     12.91186255|      77.62604769|
+----------------+-----------------+
only showing top 2 rows



11797

In [23]:
bus_stop_truncated_df = bus_stop_distinct_df.withColumn("lat_trunc", F.round("latitude_current",3)) 
bus_stop_truncated_df = bus_stop_truncated_df.withColumn("long_trunc", F.round("longitude_current",3)) 

In [24]:
bus_stop_truncated_df.show(2)

+----------------+-----------------+---------+----------+
|latitude_current|longitude_current|lat_trunc|long_trunc|
+----------------+-----------------+---------+----------+
|     12.84306239|       77.7275245|   12.843|    77.728|
|     13.23076315|      77.55015835|   13.231|     77.55|
+----------------+-----------------+---------+----------+
only showing top 2 rows



In [25]:
bus_stop_truncated_distinct_df = bus_stop_truncated_df.select("lat_trunc","long_trunc").distinct()
bus_stop_truncated_distinct_count = bus_stop_truncated_distinct_df.count()
bus_stop_truncated_distinct_count

6722

## Create a "fence" around each bus stop so that extreme truncated values are accommodated

In [26]:
bus_stop_augmented_df = bus_stop_truncated_distinct_df.withColumn("lat_neg_extended",
                                                                  F.round(bus_stop_truncated_distinct_df.lat_trunc - 0.0007, 4))
bus_stop_augmented_df = bus_stop_augmented_df.withColumn("lat_pos_extended",
                                                         F.round(bus_stop_augmented_df.lat_trunc + 0.0007,4))
bus_stop_augmented_df = bus_stop_augmented_df.withColumn("long_neg_extended",
                                                         F.round(bus_stop_augmented_df.long_trunc - 0.0007, 4))
bus_stop_augmented_df = bus_stop_augmented_df.withColumn("long_pos_extended",
                                                         F.round(bus_stop_augmented_df.long_trunc + 0.0007, 4))

In [27]:
bus_stop_augmented_df.show(2)

+---------+----------+----------------+----------------+-----------------+-----------------+
|lat_trunc|long_trunc|lat_neg_extended|lat_pos_extended|long_neg_extended|long_pos_extended|
+---------+----------+----------------+----------------+-----------------+-----------------+
|   13.095|      77.5|         13.0943|         13.0957|          77.4993|          77.5007|
|   12.983|    77.589|         12.9823|         12.9837|          77.5883|          77.5897|
+---------+----------+----------------+----------------+-----------------+-----------------+
only showing top 2 rows



In [28]:
bus_stop_augmented_select_df = bus_stop_augmented_df.select("lat_neg_extended","lat_pos_extended",
                                                            "long_neg_extended","long_pos_extended")
bus_stop_augmented_select_count = bus_stop_augmented_select_df.count()
bus_stop_augmented_select_count

6722

##  Keep those rows of VTS which fall within the grid around the bus stop

In [29]:
# Define the joining condition
join_condition = [vts_volvo_df.lat >= bus_stop_augmented_select_df.lat_neg_extended,
                  vts_volvo_df.lat <= bus_stop_augmented_select_df.lat_pos_extended,
                  vts_volvo_df.longitude >= bus_stop_augmented_select_df.long_neg_extended,
                  vts_volvo_df.longitude <= bus_stop_augmented_select_df.long_pos_extended]

In [30]:
vts_bus_stop_joined_df = vts_volvo_df.join(broadcast(bus_stop_augmented_select_df), join_condition, "inner")

In [31]:
vts_bus_stop_joined_df.show(2)

+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+----------------+----------------+-----------------+-----------------+
|         id|VTS_device_id|ign_status|acc_distance|            ist_date|      lat|longitude|vehicle_direction|lat_neg_extended|lat_pos_extended|long_neg_extended|long_pos_extended|
+-----------+-------------+----------+------------+--------------------+---------+---------+-----------------+----------------+----------------+-----------------+-----------------+
|12132590069|    150221435|         0|    44149312|2017-01-01 02:09:...|13.004257|77.692299|            188.0|         13.0033|         13.0047|          77.6913|          77.6927|
|12132590072|    150221408|         0|    47214336|2017-01-01 02:09:...|12.977072| 77.72702|            118.0|         12.9763|         12.9777|          77.7263|          77.7277|
+-----------+-------------+----------+------------+--------------------+---------+---------+---

In [32]:
# Since a bus can be closer to be multiple bus stops, hence joining operation inflates the entries in the VTS data. 
vts_bus_stop_joined_distinct_df = vts_bus_stop_joined_df.select("id", "VTS_device_id", 
                                                                "ist_date","lat","longitude").distinct()

In [33]:
# vts_bus_stop_joined_distinct_count = vts_bus_stop_joined_distinct_df.count()
# vts_bus_stop_joined_distinct_count

In [ ]:
sqlContext.sql("use bmtc_eta_default")

DataFrame[]

In [ ]:
# Instead of creating a persistent table using saveAsTable, make temp table and dump it as a hive table
vts_bus_stop_joined_distinct_df.createOrReplaceTempView("temp_vts_bus_stop_joined_distinct_df") 
sqlContext.sql("create table vts_volvo_2017 as select * from temp_vts_bus_stop_joined_distinct_df")

In [37]:
# SUCCESS!
sqlContext.sql("show tables").show()

+--------------------+-----------+
|           tableName|isTemporary|
+--------------------+-----------+
|            vts_365r|      false|
|   vts_365r_distance|      false|
| vts_365r_distance_1|      false|
|   vts_365r_filtered|      false|
|         vts_jun2017|      false|
|      vts_volvo_2017|      false|
|   vts_volvo_oct2017|      false|
|temp_vts_bus_stop...|       true|
+--------------------+-----------+



In [38]:
sqlContext.sql("select count(*) from vts_volvo_2017").show()

+---------+
| count(1)|
+---------+
|956091019|
+---------+

